In [1]:
import os
import glob
import shutil
import random
import time
from dataclasses import dataclass

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils.class_weight import compute_class_weight

import cv2
from PIL import Image

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

@dataclass
class CFG:
    DATASET_DIR: str = "/kaggle/input/lab6-spotify-dataset/dataset"

    WORK_RAW_DIR: str = "/kaggle/working/raw_dataset"
    WORK_AUG_DIR: str = "/kaggle/working/augmented_dataset"
    WORK_SPLIT_DIR: str = "/kaggle/working/split_dataset"

    TARGET_SIZE: tuple = (299, 299)
    BATCH_SIZE: int = 16

    SPLIT_RATIOS: tuple = (0.7, 0.15, 0.15)

    OFFLINE_AUG_MAX_PER_IMAGE: int = 5
    OFFLINE_AUG_FILL_MODE: str = "nearest"

    ONLINE_AUG: bool = True

    EPOCHS: int = 30
    LR_FROZEN: float = 1e-3
    LR_FINE: float = 1e-4
    PATIENCE: int = 6
    FROZEN_EPOCHS: int = 5

    IMAGENET_WEIGHTS_PATH: str = (
        "/kaggle/input/xception_weights_tf_dim_ordering_tf_kernels_notop/"
        "tensorflow2/default/1/xception_weights_tf_dim_ordering_tf_kernels_notop.h5"
    )

    VIDEO_PATH: str = "/kaggle/input/spoti-detection/spotify-video-for-detection.mp4"
    VIDEO_BATCH: int = 32
    VIDEO_SKIP_FRAMES: int = 3
    VIDEO_THRESHOLD: float = 0.5
    SMOOTH_WINDOW: int = 7
    MIN_INTERVAL_SEC: float = 0.3


CFG = CFG()

def xception_preprocess(x):
    return (x / 127.5) - 1.0

def find_binary_dataset_dir():
    candidates = []
    for p in glob.glob("/kaggle/input/**", recursive=True):
        if os.path.isdir(p) and os.path.isdir(os.path.join(p, "positive")) and os.path.isdir(os.path.join(p, "negative")):
            candidates.append(p)
    candidates.sort(key=lambda x: (0 if os.path.basename(x).lower() == "dataset" else 1, len(x)))
    return candidates[0] if candidates else None

def clean_corrupted_images(root_dir):
    exts = (".jpg", ".jpeg", ".png", ".webp", ".bmp", ".gif", ".tiff")
    removed = 0
    total = 0
    for sub in ["positive", "negative"]:
        d = os.path.join(root_dir, sub)
        if not os.path.isdir(d):
            continue
        for fn in os.listdir(d):
            if not fn.lower().endswith(exts):
                continue
            total += 1
            fp = os.path.join(d, fn)
            try:
                with Image.open(fp) as img:
                    img.verify()
                with Image.open(fp) as img:
                    img = img.convert("RGB")
                    img.save(fp)
            except Exception:
                try:
                    os.remove(fp)
                    removed += 1
                except Exception:
                    pass
    return total, removed

# -------------------------
# Offline augmentation (balance classes BEFORE training)
# -------------------------
def offline_augment_to_balance(src_dir, out_dir, target_size=(299, 299), max_per_image=5):
    if os.path.exists(out_dir):
        shutil.rmtree(out_dir)
    os.makedirs(os.path.join(out_dir, "positive"), exist_ok=True)
    os.makedirs(os.path.join(out_dir, "negative"), exist_ok=True)

    # Copy originals
    for cls in ["positive", "negative"]:
        shutil.copytree(os.path.join(src_dir, cls), os.path.join(out_dir, cls), dirs_exist_ok=True)

    def count_images(cls):
        exts = (".jpg", ".jpeg", ".png", ".webp", ".bmp")
        d = os.path.join(out_dir, cls)
        return len([f for f in os.listdir(d) if f.lower().endswith(exts)])

    pos_n = count_images("positive")
    neg_n = count_images("negative")
    print(f"[INFO] Before offline augmentation: pos={pos_n}, neg={neg_n}")

    if pos_n == neg_n:
        print("[INFO] Classes already balanced. Skipping offline augmentation.")
        return

    minority = "positive" if pos_n < neg_n else "negative"
    need = abs(pos_n - neg_n)
    print(f"[INFO] Offline augmenting minority='{minority}' to add ~{need} images")

    aug_gen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        zoom_range=0.2,
        horizontal_flip=True,
        brightness_range=[0.8, 1.2],
        fill_mode=CFG.OFFLINE_AUG_FILL_MODE
    )

    exts = (".jpg", ".jpeg", ".png", ".webp", ".bmp")
    in_dir = os.path.join(out_dir, minority)
    src_files = [f for f in os.listdir(in_dir) if f.lower().endswith(exts)]
    if not src_files:
        raise RuntimeError(f"No images found in class '{minority}' for offline augmentation")

    made = 0
    i = 0
    while made < need:
        fn = src_files[i % len(src_files)]
        i += 1

        img_path = os.path.join(in_dir, fn)
        try:
            img = load_img(img_path, target_size=target_size)
            x = img_to_array(img)
            x = np.expand_dims(x, 0)
        except Exception:
            continue

        base = os.path.splitext(fn)[0]
        j = 0
        for batch in aug_gen.flow(x, batch_size=1):
            out_name = f"{base}_aug_{made:06d}.jpg"
            out_path = os.path.join(in_dir, out_name)
            tf.keras.preprocessing.image.array_to_img(batch[0]).save(out_path)
            made += 1
            j += 1
            if made >= need or j >= max_per_image:
                break

    pos_n2 = count_images("positive")
    neg_n2 = count_images("negative")
    print(f"[INFO] After offline augmentation: pos={pos_n2}, neg={neg_n2}")

# -------------------------
# Build split dataset: train/val/test
# -------------------------
def make_splits(src_dir, out_dir, ratios=(0.7, 0.15, 0.15), seed=42):
    if os.path.exists(out_dir):
        shutil.rmtree(out_dir)

    for split in ["train", "val", "test"]:
        for cls in ["positive", "negative"]:
            os.makedirs(os.path.join(out_dir, split, cls), exist_ok=True)

    exts = (".jpg", ".jpeg", ".png", ".webp", ".bmp")
    rng = random.Random(seed)

    for cls in ["positive", "negative"]:
        files = [f for f in os.listdir(os.path.join(src_dir, cls)) if f.lower().endswith(exts)]
        rng.shuffle(files)

        n = len(files)
        n_train = int(n * ratios[0])
        n_val = int(n * ratios[1])

        train_files = files[:n_train]
        val_files = files[n_train:n_train + n_val]
        test_files = files[n_train + n_val:]

        for f in train_files:
            shutil.copy2(os.path.join(src_dir, cls, f), os.path.join(out_dir, "train", cls, f))
        for f in val_files:
            shutil.copy2(os.path.join(src_dir, cls, f), os.path.join(out_dir, "val", cls, f))
        for f in test_files:
            shutil.copy2(os.path.join(src_dir, cls, f), os.path.join(out_dir, "test", cls, f))

        print(f"[INFO] Split '{cls}': train={len(train_files)}, val={len(val_files)}, test={len(test_files)}")

# -------------------------
# Xception backbone
# -------------------------
def build_xception_backbone_exact(input_shape=(299, 299, 3)):
    img_input = layers.Input(shape=input_shape, name="input")

    # block1
    x = layers.Conv2D(32, (3, 3), strides=(2, 2), use_bias=False, padding="valid", name="block1_conv1")(img_input)
    x = layers.BatchNormalization(name="block1_conv1_bn")(x)
    x = layers.Activation("relu", name="block1_conv1_act")(x)

    x = layers.Conv2D(64, (3, 3), use_bias=False, padding="valid", name="block1_conv2")(x)
    x = layers.BatchNormalization(name="block1_conv2_bn")(x)
    x = layers.Activation("relu", name="block1_conv2_act")(x)

    # block2
    residual = layers.Conv2D(128, (1, 1), strides=(2, 2), padding="same", use_bias=False, name="block2_res_conv")(x)
    residual = layers.BatchNormalization(name="block2_res_conv_bn")(residual)

    x = layers.SeparableConv2D(128, (3, 3), padding="same", use_bias=False, name="block2_sepconv1")(x)
    x = layers.BatchNormalization(name="block2_sepconv1_bn")(x)
    x = layers.Activation("relu", name="block2_sepconv1_act")(x)

    x = layers.SeparableConv2D(128, (3, 3), padding="same", use_bias=False, name="block2_sepconv2")(x)
    x = layers.BatchNormalization(name="block2_sepconv2_bn")(x)

    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding="same", name="block2_pool")(x)
    x = layers.Add(name="block2_add")([x, residual])

    # block3
    residual = layers.Conv2D(256, (1, 1), strides=(2, 2), padding="same", use_bias=False, name="block3_res_conv")(x)
    residual = layers.BatchNormalization(name="block3_res_conv_bn")(residual)

    x = layers.Activation("relu", name="block3_sepconv1_act")(x)
    x = layers.SeparableConv2D(256, (3, 3), padding="same", use_bias=False, name="block3_sepconv1")(x)
    x = layers.BatchNormalization(name="block3_sepconv1_bn")(x)

    x = layers.Activation("relu", name="block3_sepconv2_act")(x)
    x = layers.SeparableConv2D(256, (3, 3), padding="same", use_bias=False, name="block3_sepconv2")(x)
    x = layers.BatchNormalization(name="block3_sepconv2_bn")(x)

    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding="same", name="block3_pool")(x)
    x = layers.Add(name="block3_add")([x, residual])

    # block4
    residual = layers.Conv2D(728, (1, 1), strides=(2, 2), padding="same", use_bias=False, name="block4_res_conv")(x)
    residual = layers.BatchNormalization(name="block4_res_conv_bn")(residual)

    x = layers.Activation("relu", name="block4_sepconv1_act")(x)
    x = layers.SeparableConv2D(728, (3, 3), padding="same", use_bias=False, name="block4_sepconv1")(x)
    x = layers.BatchNormalization(name="block4_sepconv1_bn")(x)

    x = layers.Activation("relu", name="block4_sepconv2_act")(x)
    x = layers.SeparableConv2D(728, (3, 3), padding="same", use_bias=False, name="block4_sepconv2")(x)
    x = layers.BatchNormalization(name="block4_sepconv2_bn")(x)

    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding="same", name="block4_pool")(x)
    x = layers.Add(name="block4_add")([x, residual])

    # middle flow blocks 5..12 (8 blocks)
    for i in range(8):
        block_id = 5 + i
        prefix = f"block{block_id}"
        residual = x

        x = layers.Activation("relu", name=f"{prefix}_sepconv1_act")(x)
        x = layers.SeparableConv2D(728, (3, 3), padding="same", use_bias=False, name=f"{prefix}_sepconv1")(x)
        x = layers.BatchNormalization(name=f"{prefix}_sepconv1_bn")(x)

        x = layers.Activation("relu", name=f"{prefix}_sepconv2_act")(x)
        x = layers.SeparableConv2D(728, (3, 3), padding="same", use_bias=False, name=f"{prefix}_sepconv2")(x)
        x = layers.BatchNormalization(name=f"{prefix}_sepconv2_bn")(x)

        x = layers.Activation("relu", name=f"{prefix}_sepconv3_act")(x)
        x = layers.SeparableConv2D(728, (3, 3), padding="same", use_bias=False, name=f"{prefix}_sepconv3")(x)
        x = layers.BatchNormalization(name=f"{prefix}_sepconv3_bn")(x)

        x = layers.Add(name=f"{prefix}_add")([x, residual])

    # exit flow: block13
    residual = layers.Conv2D(1024, (1, 1), strides=(2, 2), padding="same", use_bias=False, name="block13_res_conv")(x)
    residual = layers.BatchNormalization(name="block13_res_conv_bn")(residual)

    x = layers.Activation("relu", name="block13_sepconv1_act")(x)
    x = layers.SeparableConv2D(728, (3, 3), padding="same", use_bias=False, name="block13_sepconv1")(x)
    x = layers.BatchNormalization(name="block13_sepconv1_bn")(x)

    x = layers.Activation("relu", name="block13_sepconv2_act")(x)
    x = layers.SeparableConv2D(1024, (3, 3), padding="same", use_bias=False, name="block13_sepconv2")(x)
    x = layers.BatchNormalization(name="block13_sepconv2_bn")(x)

    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding="same", name="block13_pool")(x)
    x = layers.Add(name="block13_add")([x, residual])

    # block14
    x = layers.SeparableConv2D(1536, (3, 3), padding="same", use_bias=False, name="block14_sepconv1")(x)
    x = layers.BatchNormalization(name="block14_sepconv1_bn")(x)
    x = layers.Activation("relu", name="block14_sepconv1_act")(x)

    x = layers.SeparableConv2D(2048, (3, 3), padding="same", use_bias=False, name="block14_sepconv2")(x)
    x = layers.BatchNormalization(name="block14_sepconv2_bn")(x)
    x = layers.Activation("relu", name="block14_sepconv2_act")(x)

    return models.Model(img_input, x, name="xception_backbone_custom")


def build_xception_binary(weights_path=None, input_shape=(299, 299, 3), dropout=0.5):
    backbone = build_xception_backbone_exact(input_shape=input_shape)

    if weights_path and os.path.exists(weights_path):
        backbone.load_weights(weights_path)  # load NOTOP weights into backbone
        print("[INFO] Loaded ImageNet NOTOP weights into backbone:", weights_path)
    else:
        print("[INFO] ImageNet weights not provided or not found. Training from scratch.")

    x = backbone.output
    x = layers.GlobalAveragePooling2D(name="gap")(x)
    x = layers.Dropout(dropout, name="drop")(x)
    outputs = layers.Dense(1, activation="sigmoid", name="pred")(x)

    model = models.Model(backbone.input, outputs, name="Xception_custom_bin")

    return model, backbone


# -------------------------
# Video utilities -> intervals
# -------------------------
def smooth_predictions(predictions, window_size=7, threshold=0.5):
    smoothed = []
    preds = list(map(float, predictions))
    for i in range(len(preds)):
        w = preds[max(0, i - window_size // 2): i + window_size // 2 + 1]
        smoothed.append(1 if float(np.mean(w)) >= threshold else 0)
    return smoothed

def to_intervals(times, labels01, min_len_sec=0.3):
    intervals = []
    start = None
    for t, y in zip(times, labels01):
        if y == 1 and start is None:
            start = t
        elif y == 0 and start is not None:
            end = t
            if end - start >= min_len_sec:
                intervals.append((start, end))
            start = None
    if start is not None and times:
        end = times[-1]
        if end - start >= min_len_sec:
            intervals.append((start, end))
    return intervals

def process_video_batch(video_path, model, target_size, batch_size=32, skip_frames=3):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError("[ERROR] Не вдалося відкрити відео!")

    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_probs = []
    frame_times = []

    batch = []
    batch_times = []
    frame_idx = 0
    t0 = time.time()

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_idx += 1
        if skip_frames > 1 and (frame_idx % skip_frames) != 0:
            continue

        t_sec = frame_idx / fps
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, target_size)

        # preprocess to [-1, 1]
        arr = img.astype(np.float32)
        arr = (arr / 127.5) - 1.0

        batch.append(arr)
        batch_times.append(t_sec)

        if len(batch) == batch_size:
            p = model.predict(np.array(batch), verbose=0).reshape(-1)
            frame_probs.extend(p.tolist())
            frame_times.extend(batch_times)
            batch, batch_times = [], []

    if batch:
        p = model.predict(np.array(batch), verbose=0).reshape(-1)
        frame_probs.extend(p.tolist())
        frame_times.extend(batch_times)

    cap.release()
    total_time = time.time() - t0
    return frame_probs, frame_times, total_time


# -------------------------
# Main
# -------------------------
def main():
    print("[INFO] TensorFlow:", tf.__version__)
    print("[INFO] GPU:", tf.config.list_physical_devices("GPU"))

    dataset_dir = CFG.DATASET_DIR or find_binary_dataset_dir()
    if dataset_dir is None:
        raise RuntimeError("Не знайдено dataset/{positive,negative} у /kaggle/input. Вкажіть CFG.DATASET_DIR вручну.")
    print("[INFO] Using DATASET_DIR:", dataset_dir)

    if os.path.exists(CFG.WORK_RAW_DIR):
        shutil.rmtree(CFG.WORK_RAW_DIR)
    shutil.copytree(dataset_dir, CFG.WORK_RAW_DIR)
    dataset_dir = CFG.WORK_RAW_DIR

    # clean corrupted
    total, removed = clean_corrupted_images(dataset_dir)
    print(f"[INFO] Cleaned corrupted images: removed={removed} / scanned={total}")

    offline_augment_to_balance(
        src_dir=dataset_dir,
        out_dir=CFG.WORK_AUG_DIR,
        target_size=CFG.TARGET_SIZE,
        max_per_image=CFG.OFFLINE_AUG_MAX_PER_IMAGE
    )

    # build train/val/test split
    make_splits(CFG.WORK_AUG_DIR, CFG.WORK_SPLIT_DIR, ratios=CFG.SPLIT_RATIOS, seed=SEED)

    if CFG.ONLINE_AUG:
        train_datagen = ImageDataGenerator(
            preprocessing_function=xception_preprocess,
            rotation_range=15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.1,
            zoom_range=0.1,
            horizontal_flip=True,
            brightness_range=[0.9, 1.1],
        )
    else:
        train_datagen = ImageDataGenerator(preprocessing_function=xception_preprocess)

    eval_datagen = ImageDataGenerator(preprocessing_function=xception_preprocess)

    train_gen = train_datagen.flow_from_directory(
        os.path.join(CFG.WORK_SPLIT_DIR, "train"),
        target_size=CFG.TARGET_SIZE,
        batch_size=CFG.BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED
    )
    val_gen = eval_datagen.flow_from_directory(
        os.path.join(CFG.WORK_SPLIT_DIR, "val"),
        target_size=CFG.TARGET_SIZE,
        batch_size=CFG.BATCH_SIZE,
        class_mode="binary",
        shuffle=False
    )
    test_gen = eval_datagen.flow_from_directory(
        os.path.join(CFG.WORK_SPLIT_DIR, "test"),
        target_size=CFG.TARGET_SIZE,
        batch_size=CFG.BATCH_SIZE,
        class_mode="binary",
        shuffle=False
    )

    print("[INFO] class_indices:", train_gen.class_indices)  # usually {'negative':0, 'positive':1}

    class_weights = compute_class_weight(
        class_weight="balanced",
        classes=np.unique(train_gen.classes),
        y=train_gen.classes
    )
    class_weight_dict = dict(enumerate(class_weights))
    print("[INFO] class_weight_dict:", class_weight_dict)

    model, backbone = build_xception_binary(
        weights_path=CFG.IMAGENET_WEIGHTS_PATH,
        input_shape=CFG.TARGET_SIZE + (3,),
        dropout=0.5
    )

    best_weights_path = "/kaggle/working/best_model.weights.h5"

    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=CFG.PATIENCE,
            restore_best_weights=True,
            verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=max(2, CFG.PATIENCE // 2),
            min_lr=1e-7,
            verbose=1
        ),
        tf.keras.callbacks.ModelCheckpoint(
            best_weights_path,
            monitor="val_accuracy",
            save_best_only=True,
            save_weights_only=True,
            verbose=1
        )
    ]

    history_all = {"accuracy": [], "val_accuracy": [], "loss": [], "val_loss": [], "auc": [], "val_auc": []}

    using_pretrained = CFG.IMAGENET_WEIGHTS_PATH and os.path.exists(CFG.IMAGENET_WEIGHTS_PATH)
    frozen_epochs = CFG.FROZEN_EPOCHS if using_pretrained else 0
    fine_epochs = CFG.EPOCHS - frozen_epochs

    if frozen_epochs > 0:
        print("\n" + "=" * 60)
        print("[INFO] PHASE 1: Train head only (backbone frozen)")
        print("=" * 60 + "\n")

        backbone.trainable = False
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=CFG.LR_FROZEN),
            loss="binary_crossentropy",
            metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
        )

        hist1 = model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=frozen_epochs,
            class_weight=class_weight_dict,
            callbacks=callbacks,
            verbose=1
        )
        for k in history_all:
            if k in hist1.history:
                history_all[k].extend(hist1.history[k])

    print("\n" + "=" * 60)
    print("[INFO] PHASE 2: Fine-tuning (backbone trainable)")
    print("=" * 60 + "\n")

    backbone.trainable = True
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=CFG.LR_FINE),
        loss="binary_crossentropy",
        metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
    )

    hist2 = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=frozen_epochs + fine_epochs,
        initial_epoch=frozen_epochs,
        class_weight=class_weight_dict,
        callbacks=callbacks,
        verbose=1
    )
    for k in history_all:
        if k in hist2.history:
            history_all[k].extend(hist2.history[k])

    model.save("/kaggle/working/xception_custom_binary.keras")
    model.load_weights(best_weights_path)

    # Evaluate on test
    print("[INFO] Evaluating on test split...")
    y_prob = model.predict(test_gen, verbose=1).reshape(-1)
    y_pred = (y_prob >= 0.5).astype(int)
    y_true = test_gen.classes

    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()

    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) else 0.0
    recall = tp / (tp + fn) if (tp + fn) else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0

    print("\n[INFO] Confusion matrix:\n", cm)
    print(f"[INFO] Accuracy : {accuracy:.4f}")
    print(f"[INFO] Precision: {precision:.4f}")
    print(f"[INFO] Recall   : {recall:.4f}")
    print(f"[INFO] F1-score : {f1:.4f}\n")
    print(classification_report(y_true, y_pred, target_names=["Negative", "Positive"]))

    plt.figure(figsize=(7, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=["Neg", "Pos"], yticklabels=["Neg", "Pos"])
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")
    plt.tight_layout()
    plt.savefig("/kaggle/working/confusion_matrix.png", dpi=250)
    plt.close()

    # Training curves
    plt.figure(figsize=(14, 4))
    plt.subplot(1, 3, 1)
    plt.plot(history_all.get("accuracy", []), label="train")
    plt.plot(history_all.get("val_accuracy", []), label="val")
    plt.title("Accuracy"); plt.legend(); plt.grid(alpha=0.3)

    plt.subplot(1, 3, 2)
    plt.plot(history_all.get("loss", []), label="train")
    plt.plot(history_all.get("val_loss", []), label="val")
    plt.title("Loss"); plt.legend(); plt.grid(alpha=0.3)

    plt.subplot(1, 3, 3)
    plt.plot(history_all.get("auc", []), label="train")
    plt.plot(history_all.get("val_auc", []), label="val")
    plt.title("AUC"); plt.legend(); plt.grid(alpha=0.3)

    plt.tight_layout()
    plt.savefig("/kaggle/working/training_curves.png", dpi=250)
    plt.close()

    # Video inference -> intervals
    if CFG.VIDEO_PATH and os.path.exists(CFG.VIDEO_PATH):
        print("[INFO] Video inference:", CFG.VIDEO_PATH)
        probs, times_sec, tproc = process_video_batch(
            CFG.VIDEO_PATH, model, CFG.TARGET_SIZE,
            batch_size=CFG.VIDEO_BATCH,
            skip_frames=CFG.VIDEO_SKIP_FRAMES
        )
        labels = smooth_predictions(probs, window_size=CFG.SMOOTH_WINDOW, threshold=CFG.VIDEO_THRESHOLD)
        intervals = to_intervals(times_sec, labels, min_len_sec=CFG.MIN_INTERVAL_SEC)

        print(f"[INFO] Video processed in {tproc:.2f}s, intervals found: {len(intervals)}")
        out_csv = "/kaggle/working/video_intervals.csv"
        with open(out_csv, "w", encoding="utf-8") as f:
            f.write("start_sec,end_sec\n")
            for a, b in intervals:
                f.write(f"{a:.3f},{b:.3f}\n")
        print("[INFO] Saved intervals to:", out_csv)

        plt.figure(figsize=(14, 5))
        plt.plot(times_sec, probs, label="prob", alpha=0.7)
        plt.axhline(CFG.VIDEO_THRESHOLD, color="red", linestyle="--", label="threshold")
        plt.title("Video predictions over time")
        plt.xlabel("time (sec)")
        plt.ylabel("P(positive)")
        plt.grid(alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.savefig("/kaggle/working/video_predictions.png", dpi=250)
        plt.close()
    else:
        print("[INFO] VIDEO_PATH not set or file not found. Skipping video step.")

    print("[INFO] Done. Outputs are in /kaggle/working/")

if __name__ == "__main__":
    main()


2025-12-17 23:11:27.526970: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766013087.703437      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766013087.761601      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766013088.204435      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766013088.204477      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766013088.204479      55 computation_placer.cc:177] computation placer alr

[INFO] TensorFlow: 2.19.0
[INFO] GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[INFO] Using DATASET_DIR: /kaggle/input/lab6-spotify-dataset/dataset


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


[INFO] Cleaned corrupted images: removed=0 / scanned=1368
[INFO] Before offline augmentation: pos=671, neg=697
[INFO] Offline augmenting minority='positive' to add ~26 images
[INFO] After offline augmentation: pos=697, neg=697
[INFO] Split 'positive': train=487, val=104, test=106
[INFO] Split 'negative': train=487, val=104, test=106
Found 974 images belonging to 2 classes.
Found 208 images belonging to 2 classes.
Found 212 images belonging to 2 classes.
[INFO] class_indices: {'negative': 0, 'positive': 1}
[INFO] class_weight_dict: {0: np.float64(1.0), 1: np.float64(1.0)}


I0000 00:00:1766013163.048588      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


...

58/61 ━━━━━━━━━━━━━━━━━━━━ 1s 469ms/step - accuracy: 0.9052 - auc: 0.9741 - loss: 0.2790

2025-12-17 23:17:45.058634: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-12-17 23:17:45.294270: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-12-17 23:17:48.688522: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-12-17 23:17:48.893767: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-12-17 23:17:49.931229: E external/local_xla/xla/stream_

61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9073 - auc: 0.9746 - loss: 0.2734 
Epoch 6: val_accuracy improved from 0.95192 to 0.96154, saving model to /kaggle/working/best_model.weights.h5
61/61 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - accuracy: 0.9079 - auc: 0.9748 - loss: 0.2717 - val_accuracy: 0.9615 - val_auc: 0.9982 - val_loss: 0.1165 - learning_rate: 1.0000e-04
Epoch 7/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 483ms/step - accuracy: 0.9939 - auc: 0.9994 - loss: 0.0445
Epoch 7: val_accuracy improved from 0.96154 to 0.97115, saving model to /kaggle/working/best_model.weights.h5
61/61 ━━━━━━━━━━━━━━━━━━━━ 33s 536ms/step - accuracy: 0.9939 - auc: 0.9994 - loss: 0.0444 - val_accuracy: 0.9712 - val_auc: 0.9994 - val_loss: 0.0482 - learning_rate: 1.0000e-04
Epoch 8/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 480ms/step - accuracy: 0.9981 - auc: 1.0000 - loss: 0.0106
Epoch 8: val_accuracy improved from 0.97115 to 0.97596, saving model to /kaggle/working/best_model.weights.h5
61/61 ━━━━━━━━━━━━━━━━━━━━ 33

2025-12-17 23:27:31.262821: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-12-17 23:27:31.503960: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-12-17 23:27:32.601009: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-12-17 23:27:32.866465: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


[INFO] Video processed in 40.65s, intervals found: 2
[INFO] Saved intervals to: /kaggle/working/video_intervals.csv
[INFO] Done. Outputs are in /kaggle/working/
